# This notebook uses the previously trained model for songs in range 8 and produces a playable song file from a directory

In [68]:
from pathlib import Path
import pandas as pd
import re
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
import os
import librosa
from pydub import AudioSegment
from fastai.text import *
import string

In [69]:
# Load our language model which will allow us to generate the step patterns
cwd = os.getcwd()
path = Path(cwd)

data_lm = load_data(path, 'data_block_lm4.pkl',bs=96)
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
learn.load('fine_tuned_4')
learn.load_encoder('fine_tuned_enc_4')

In [70]:
def GetFileName(filePath):
    return ''.join(filePath.name.split('.')[:-1]).lstrip()

def RenameFiles(basePath):
    """Turn File name to All Uppercase and remove characters that are not letters or numbers"""
    allowableCharacters = string.ascii_letters + ' '
    
    #Folder of music
    mp3Files = list(basePath.rglob("*.[mM][pP][3]"))

    for mp3 in mp3Files:    
        #Old name without mp3 extension
        oldName = GetFileName(mp3)
        #Only characters in allowable list and then made into uppercase
        newName = ''.join([i for i in oldName if i in allowableCharacters]).upper()
        #Add on mp3 extension
        newName = newName + ".mp3"
        os.rename(mp3, basePath/newName)
    
    return 0

def MakeSongImage(filePath, songTitle, fontSize = 175):
    """Take a image and write the song title on it in white text of size 175 font in the top left corner"""
    img = Image.open(filePath)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype("times.ttf", fontSize)
    W, H = img.size
    w, h = draw.textsize(songTitle)

    draw.text(((W-w)/2,(H-h)/2),songTitle,(255,255,255),font=font)
    
    return img

def MakeMP3(filePath, shorten = True, shortenLength = 90000):
    """Shorten the mp3 if it is greater than the length specified.  Default is 90 seconds"""
    
    song = AudioSegment.from_mp3(filePath)
    newSong = song
    
    if shorten and (len(song) > shortenLength):
        newSong = song[:shortenLength]    
    
    return newSong

def MakeDWI(filePath, title, bpm, gap, predictedSteps, artist="Deep Learning", sampleStart = 30):
    """Create the DWI File for each song from the parameters passed to the function"""
    #cdtitle = "#CDTITLE:.\\CDTITLES\\" + mixTitle + ".PNG;"
    title = "#TITLE:"+title+";\n"
    artist = "#ARTIST:"+artist+";\n"
    bpm = "#BPM:"+str(bpm)+";\n"
    gap = "#GAP:"+str(gap)+";\n"
    sampleStart = "#SAMPLESTART:"+str(sampleStart)+";\n\n"
    steps = "#SINGLE:MANIAC:8:"+ predictedSteps + ";\n"
    
    #Write the file if it doesn't exist already
    if not os.path.exists(filePath):
        with open(str(filePath), "w") as dwiFile:
            dwiFile.write(title + artist + bpm + gap + sampleStart + steps)
    
    return 0

def MakeSteps(numberBeats = 800, temperature = 1.0):
    """Generate steps for a stepmania song"""
    
    songSteps = ""
    predictedSteps = "".join(learn.predict(songSteps, numberBeats, temperature=temperature))
    
    #Remove spaces from characters
    predictedSteps = predictedSteps.replace(" ","")
    
    #Remove beginning of sentence tokens if they occur
    predictedSteps = predictedSteps.replace("xxbos","")
    
    return predictedSteps

def GetGapTime(beatTimes, gapLength = 200):
    """Gets the first beat after X milliseconds, 200 by default"""
    for beatTime in beatTimes:
        if beatTime > gapLength:
            return beatTime
        
def GetBPM(fileName):
    # Load the audio as a waveform `y`, Store the sampling rate as `sr`
    y, sr = librosa.load(fileName)
    tempo, beatFrames = librosa.beat.beat_track(y=y, sr=sr)

    # Convert the frame indices of beat events into timestamps
    beatTimes = librosa.frames_to_time(beatFrames, sr=sr)
    #Convert to milliseconds
    beatTimes = beatTimes * 1000
    
    gap = GetGapTime(beatTimes)
    bpm = tempo
    
    return (bpm, gap)

In [71]:
#Title of the mix for the Step Mania Directory
mixTitle = "Brent Mix"

#Base Path for new Directory
basePath = Path("C:/Users/brent/Desktop/input")

#Rename the files in the directory for consistency and remove strange characters
RenameFiles(basePath)   

#Folder of music
mp3Files = list(basePath.rglob("*.[mM][pP][3]"))

#Assume there is a single background image
backgroundImage = list(basePath.rglob("*.[jJ][pP][gG]"))[0]

# Code block that generates the DWI files and places everything in directories

In [ ]:
#Make the Base folder for the mix
if not os.path.exists(basePath/mixTitle):
        os.mkdir(basePath/mixTitle)
     
for mp3 in mp3Files:
    #Just extract the name of the file without extension
    songTitle = GetFileName(mp3)
    
    #Make folder for each song within the mix directory
    if not os.path.exists(basePath/mixTitle/songTitle):
        os.mkdir(basePath/mixTitle/songTitle)
    
    #Create the background image file for each folder
    songImage = MakeSongImage(backgroundImage, songTitle)
    songImage.save(basePath/mixTitle/songTitle/str(songTitle+'-bg.png'))
    
    #Copy the mp3 file and shorten if desired
    mp3File = MakeMP3(mp3,shorten=True,shortenLength=200000)
    mp3File.export(basePath/mixTitle/songTitle/str(songTitle+'.mp3'), format="mp3")
    
    #Song duration in milliseconds
    songDuration = len(mp3File)
    
    #Generate the step pattern
    bpm, gap = GetBPM(basePath/mixTitle/songTitle/str(songTitle+'.mp3'))
    
    #Formula to determine the number of beats:
    beatFactor = 1.75 #Scaling factor chosen based on experimentation
    numberBeats = math.floor(((((songDuration - gap) / 1000) / 60) * bpm) * beatFactor)
    
    stepPattern = MakeSteps(numberBeats = numberBeats, temperature = 0.9)
    
    #Write the DWI File
    MakeDWI(basePath/mixTitle/songTitle/str(songTitle+'.dwi'), songTitle, bpm, gap, stepPattern)

C:\Users\brent\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\fftpack\basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x
C:\Users\brent\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\fftpack\basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x
C:\Users\brent\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\fftpack\basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the fu